In [1]:
import pandas as pd
import re
import numpy as np
import os
import glob

In [2]:
import configparser

config = configparser.RawConfigParser()
config.read('parametros.config')

PARAMETROS = dict(config.items('Padronizacao_conversao'))
PARAMETROS_GERAIS = dict(config.items('Geral'))

In [3]:
regex = {}
regex['ATIVO']                          = {'regex':  "1            "}
regex['ATIVO_CIRCULANTE']               = {'regex':  "12           "}
regex['CREDITO_OPER_PS']                = {'regex':  "123          "}
regex['PASSIVO']                        = {'regex':  "2            "}
regex['PASSIVO_CIRCULANTE']             = {'regex':  "21           "}
regex['PASSIVO_NAO_CIRCULANTE']         = {'regex':  "23           "}
regex['PL']                             = {'regex':  "25           "}
regex['RECEITA_COM_ASSISTENCIA_A_SAUDE']= {'regex':  "31           "}
regex['CONTRAPRESTACAO_PRE']            = {'regex':  "311[^7].1       "}
regex['CONTRAPRESTACAO_POS']            = {'regex':  "311[^7].2       "}
regex['CORRESP_CEDIDA_PRE']             = {'regex':  "0",
                                              '2018':"3117.1       ",
                                              '2019':"3117.1       "}
regex['CORRESP_CEDIDA_POS']             = {'regex':  "0",
                                              '2018':"3117.2       ",
                                              '2019':"3117.2       "}
regex['EVENTOS_INDENIZAVEIS_LIQUIDOS']  = {'regex':  "41           "}
regex['EVENTOS_INDENIZAVEIS_PRE']       = {'regex':  "411..1       "}
regex['EVENTOS_INDENIZAVEIS_POS']       = {'regex':  "411..2       "}
regex['VARIACAO_PEONA']                 = {'regex':  "414          "}
regex['DESPESA_COM']                    = {'regex':  "43           "}
regex['DESPESA_ADM']                    = {'regex':  "46           "}
regex['PROVISOES_TECNICAS']             = {'regex' : "2[1|3]1          "}
regex['PPCNG']                          = {'regex' : "2[1|3]1....11    "}
regex['REMISSAO']                       = {'regex' : "2[1|3]1....1[3|4]    "}
regex['PESL_SUS']                       = {'regex' : "2[1|3]1....2     "}
regex['PESL_OUTROS']                    = {'regex' : "2[1|3]1....3     "}
regex['PEONA_OUTROS']                   = {'regex' : "2[1|3]1....41    "}
regex['IMPOSTO SOBRE LUCRO']            = {'regex':  "61           "}
regex['TRIBUTOS DIRETOS']               = {'regex':  "32           "}
regex['RECEITA']                        = {'regex':  "3            "}
regex['DESPESA']                        = {'regex':  "4            "}
regex['TRIBUTOS DIRETOS']               = {'regex':  "32           "}
regex['RECEITA']                        = {'regex':  "3            "}
regex['DESPESA']                        = {'regex':  "4            "}
regex['OUTRAS DESPESAS OPERACIONAIS']   = {'regex':  "44           "}
regex['OUTRAS RECEITAS OPERACIONAIS']   = {'regex':  "33           "}
regex['DESPESAS FINANCEIRAS']           = {'regex':  "45           "}
regex['RECEITAS FINANCEIRAS']           = {'regex':  "35           "}
regex['PPSC']                           = {'regex':  "4419         "}
regex['APLICAÇÕES GARANTIDORAS']        = {'regex':  "1221         "}
regex['PARTICIPAÇÕES COOPERATIVAS']     = {'regex':  "132819112    "}
regex['PARTICIPAÇÕES INSTITUIÇÕES REGULADAS'] = {'regex':  "13214        "}
regex['PARTICIPAÇÕES OPS']              = {'regex':  "13211        "}
regex['CRÉDITO TRIBUTÁRIO']             = {'regex':  "1316         "}
regex['DESPESAS ANTECIPADAS']           = {'regex':  "128          "}
regex['ATIVO NÃO CIRCULANTE INTANGÍVEL']= {'regex':  "1341         "}
regex['DESPESA DE COMERCIALIZAÇÃO DIFERIDA']   = {'regex':  "1[2|3]51         "}



In [4]:
PASTA_ENTRADA = PARAMETROS_GERAIS['pasta_raiz_dados']+PARAMETROS['pasta_arquivos_contenados']

In [5]:
lista_arquivos = [i for i in glob.glob(PASTA_ENTRADA+"/*")]

In [6]:
lista_arquivos

['/opt/DADOS_ANS5/todas_demonstracoes_por_ano/2016_todas_demonstracoes_concatenadas.csv',
 '/opt/DADOS_ANS5/todas_demonstracoes_por_ano/2018_todas_demonstracoes_concatenadas.csv',
 '/opt/DADOS_ANS5/todas_demonstracoes_por_ano/2013_todas_demonstracoes_concatenadas.csv',
 '/opt/DADOS_ANS5/todas_demonstracoes_por_ano/2014_todas_demonstracoes_concatenadas.csv',
 '/opt/DADOS_ANS5/todas_demonstracoes_por_ano/2017_todas_demonstracoes_concatenadas.csv',
 '/opt/DADOS_ANS5/todas_demonstracoes_por_ano/2015_todas_demonstracoes_concatenadas.csv',
 '/opt/DADOS_ANS5/todas_demonstracoes_por_ano/2019_todas_demonstracoes_concatenadas.csv']

In [7]:
def ordenar_dataset(data):
    return data.sort_values(by=["REG_ANS",'ANO','TRIMESTRE',"CD_CONTA_CONTABIL"])

In [8]:
def ajustar_linhas(data):
    data['CD_CONTA_CONTABIL'] = data['CD_CONTA_CONTABIL'].str.ljust(width=13, fillchar=' ')
    data['VL_SALDO_FINAL'] = data['VL_SALDO_FINAL'].str.replace(pat=',', repl='.').astype(float)
    data = ordenar_dataset(data)
    return data

In [9]:
def process(item,data,_param_conta):
    valor_atual = item['VL_SALDO_FINAL']
    item[_nome_conta] = str(float("{0:.2f}".format((valor_atual))))
    return item    

In [10]:
def ajustar_contas(_nome_conta, _param_conta, data, lista_codigos):
    _ano = data['ANO'].unique().tolist()[0]
    _regex = _param_conta['regex']
    if _ano in _param_conta:
        _regex = _param_conta[_ano]
    print(_ano,_nome_conta,"-->"+_regex+"<--")
    r = re.compile(_regex)
    lista_match_regex = list(filter(r.match, lista_codigos)) 
    df_contas = data[data['CD_CONTA_CONTABIL'].isin(lista_match_regex)]
#     print(df_contas)
    df_contas.insert(loc=0, column='pos', value=np.arange(len(df_contas)))
    df_contas = df_contas.apply(lambda x : process(x,df_contas,_param_conta),axis=1)
    df_contas.drop(columns=['pos'],inplace=True)
    return df_contas

In [11]:
PASTA_DEMONSTRACOES_CONTAS = PARAMETROS_GERAIS['pasta_raiz_dados']+os.path.sep+"DEMONSTRACOES_CONTAS"
PASTA_DEMONSTRACOES_CONTAS_SUMARIZADAS = PARAMETROS_GERAIS['pasta_raiz_dados']+os.path.sep+"DEMONSTRACOES_CONTAS_SUMARIZADAS"
if (not(os.path.exists(PASTA_DEMONSTRACOES_CONTAS))):
    os.mkdir(PASTA_DEMONSTRACOES_CONTAS)
if (not(os.path.exists(PASTA_DEMONSTRACOES_CONTAS_SUMARIZADAS))):
    os.mkdir(PASTA_DEMONSTRACOES_CONTAS_SUMARIZADAS)

In [12]:
for _arquivo in lista_arquivos:
    ano_arquivo  = _arquivo.split(os.path.sep)[-1].split("_")[0]
    data = pd.read_csv(_arquivo, dtype=str,encoding="ISO-8859-1")
    print(_arquivo)
    data = data.set_index("CHAVE")
    data = ajustar_linhas(data)
    lista_codigos_conta_contabil = data['CD_CONTA_CONTABIL'].unique().tolist()    
    for _nome_conta, _param_conta in regex.items():
        data.insert(loc=3, column=_nome_conta, value=0.0)
        df_contas = ajustar_contas(_nome_conta,_param_conta,data,lista_codigos_conta_contabil)
        data.drop(df_contas.index,inplace=True)
        data = pd.concat([data,df_contas])
    nome_arquivo = _arquivo[_arquivo.rfind(os.path.sep)+1:]
    print(nome_arquivo)
    data = ordenar_dataset(data)
    data.to_csv(PASTA_DEMONSTRACOES_CONTAS+os.path.sep+ano_arquivo+".csv",encoding="ISO-8859-1")

/opt/DADOS_ANS5/todas_demonstracoes_por_ano/2016_todas_demonstracoes_concatenadas.csv
2016 ATIVO -->1            <--
2016 ATIVO_CIRCULANTE -->12           <--
2016 CREDITO_OPER_PS -->123          <--
2016 PASSIVO -->2            <--
2016 PASSIVO_CIRCULANTE -->21           <--
2016 PASSIVO_NAO_CIRCULANTE -->23           <--
2016 PL -->25           <--
2016 RECEITA_COM_ASSISTENCIA_A_SAUDE -->31           <--
2016 CONTRAPRESTACAO_PRE -->311[^7].1       <--
2016 CONTRAPRESTACAO_POS -->311[^7].2       <--
2016 CORRESP_CEDIDA_PRE -->0<--
2016 CORRESP_CEDIDA_POS -->0<--
2016 EVENTOS_INDENIZAVEIS_LIQUIDOS -->41           <--
2016 EVENTOS_INDENIZAVEIS_PRE -->411..1       <--
2016 EVENTOS_INDENIZAVEIS_POS -->411..2       <--
2016 VARIACAO_PEONA -->414          <--
2016 DESPESA_COM -->43           <--
2016 DESPESA_ADM -->46           <--
2016 PROVISOES_TECNICAS -->2[1|3]1          <--
2016 PPCNG -->2[1|3]1....11    <--
2016 REMISSAO -->2[1|3]1....1[3|4]    <--
2016 PESL_SUS -->2[1|3]1....2     <-

### Salvando o arquivo

In [13]:
lista_arquivos = [i for i in glob.glob(PASTA_DEMONSTRACOES_CONTAS+os.path.sep+"*")]

In [14]:
lista_arquivos

['/opt/DADOS_ANS5//DEMONSTRACOES_CONTAS/2015.csv',
 '/opt/DADOS_ANS5//DEMONSTRACOES_CONTAS/2013.csv',
 '/opt/DADOS_ANS5//DEMONSTRACOES_CONTAS/2019.csv',
 '/opt/DADOS_ANS5//DEMONSTRACOES_CONTAS/2014.csv',
 '/opt/DADOS_ANS5//DEMONSTRACOES_CONTAS/2017.csv',
 '/opt/DADOS_ANS5//DEMONSTRACOES_CONTAS/2016.csv',
 '/opt/DADOS_ANS5//DEMONSTRACOES_CONTAS/2018.csv']

In [15]:
def converter_colunas_regex_para_float(df):
    for coluna in list(regex.keys()):
        df[coluna] = df[coluna].astype(float)

In [16]:
for arquivo in lista_arquivos:
    nome_arquivo = arquivo.split(os.path.sep)[-1]
    print(nome_arquivo)
    df_temp = pd.read_csv(arquivo,dtype=str,encoding="ISO-8859-1")
    converter_colunas_regex_para_float(df_temp)
#     df_temp = df_temp[colunas_dataset]
    print(df_temp.columns)
    df_temp = df_temp.groupby(["REG_ANS","ANO","TRIMESTRE","PERIODO"],as_index=False).sum()
    df_temp.to_csv(PASTA_DEMONSTRACOES_CONTAS_SUMARIZADAS+os.path.sep+nome_arquivo,index=False,encoding="ISO-8859-1")
    print("Salvo")

2015.csv
Index(['Unnamed: 0', 'PERIODO', 'ANO', 'TRIMESTRE',
       'DESPESA DE COMERCIALIZAÇÃO DIFERIDA',
       'ATIVO NÃO CIRCULANTE INTANGÍVEL', 'DESPESAS ANTECIPADAS',
       'CRÉDITO TRIBUTÁRIO', 'PARTICIPAÇÕES OPS',
       'PARTICIPAÇÕES INSTITUIÇÕES REGULADAS', 'PARTICIPAÇÕES COOPERATIVAS',
       'APLICAÇÕES GARANTIDORAS', 'PPSC', 'RECEITAS FINANCEIRAS',
       'DESPESAS FINANCEIRAS', 'OUTRAS RECEITAS OPERACIONAIS',
       'OUTRAS DESPESAS OPERACIONAIS', 'DESPESA', 'RECEITA',
       'TRIBUTOS DIRETOS', 'IMPOSTO SOBRE LUCRO', 'PEONA_OUTROS',
       'PESL_OUTROS', 'PESL_SUS', 'REMISSAO', 'PPCNG', 'PROVISOES_TECNICAS',
       'DESPESA_ADM', 'DESPESA_COM', 'VARIACAO_PEONA',
       'EVENTOS_INDENIZAVEIS_POS', 'EVENTOS_INDENIZAVEIS_PRE',
       'EVENTOS_INDENIZAVEIS_LIQUIDOS', 'CORRESP_CEDIDA_POS',
       'CORRESP_CEDIDA_PRE', 'CONTRAPRESTACAO_POS', 'CONTRAPRESTACAO_PRE',
       'RECEITA_COM_ASSISTENCIA_A_SAUDE', 'PL', 'PASSIVO_NAO_CIRCULANTE',
       'PASSIVO_CIRCULANTE', 'PASSIVO'

In [17]:
lista_arquivos = [i for i in glob.glob(PASTA_DEMONSTRACOES_CONTAS_SUMARIZADAS+os.path.sep+"*")]

In [18]:
lista_arquivos

['/opt/DADOS_ANS5//DEMONSTRACOES_CONTAS_SUMARIZADAS/2015.csv',
 '/opt/DADOS_ANS5//DEMONSTRACOES_CONTAS_SUMARIZADAS/2013.csv',
 '/opt/DADOS_ANS5//DEMONSTRACOES_CONTAS_SUMARIZADAS/2019.csv',
 '/opt/DADOS_ANS5//DEMONSTRACOES_CONTAS_SUMARIZADAS/2014.csv',
 '/opt/DADOS_ANS5//DEMONSTRACOES_CONTAS_SUMARIZADAS/2017.csv',
 '/opt/DADOS_ANS5//DEMONSTRACOES_CONTAS_SUMARIZADAS/2016.csv',
 '/opt/DADOS_ANS5//DEMONSTRACOES_CONTAS_SUMARIZADAS/2018.csv']

In [19]:
df_demonstracoes_concatenadas2 = pd.concat([pd.read_csv(f,dtype=str,encoding="ISO-8859-1") for f in lista_arquivos],sort=False)

In [20]:
df_demonstracoes_concatenadas2.to_csv(PARAMETROS_GERAIS['pasta_raiz_dados']+PARAMETROS_GERAIS['arquivo_final_todas_demonstracoes'],encoding="ISO-8859-1",index=False)

In [21]:
df_demonstracoes_concatenadas2.head()

,REG_ANS,ANO,TRIMESTRE,PERIODO,DESPESA DE COMERCIALIZAÇÃO DIFERIDA,ATIVO NÃO CIRCULANTE INTANGÍVEL,DESPESAS ANTECIPADAS,CRÉDITO TRIBUTÁRIO,PARTICIPAÇÕES OPS,PARTICIPAÇÕES INSTITUIÇÕES REGULADAS,...,CONTRAPRESTACAO_POS,CONTRAPRESTACAO_PRE,RECEITA_COM_ASSISTENCIA_A_SAUDE,PL,PASSIVO_NAO_CIRCULANTE,PASSIVO_CIRCULANTE,PASSIVO,CREDITO_OPER_PS,ATIVO_CIRCULANTE,ATIVO
0,300136,2015,1,201501,0.0,48495.29,160608.25,0.0,0.0,0.0,...,0.0,16839764.65,16839534.33,9897276.32,11841245.42,23903002.46,45641524.2,3381214.87,23392037.55,45641524.2
1,300136,2015,2,201502,0.0,41077.19,131979.09,0.0,0.0,0.0,...,0.0,36277753.089999996,36186848.12,10667692.81,11714472.71,26845389.32,49227554.84,3752622.61,26781779.5,49227554.84
2,300136,2015,3,201503,0.0,33659.09,187360.57,0.0,0.0,0.0,...,0.0,54804436.11,54758472.51,10759908.8,11417152.92,24767448.29,46944510.01,2430121.78,24336351.03,46944510.01
3,300136,2015,4,201504,0.0,26240.99,152494.75,0.0,0.0,0.0,...,0.0,74986134.66,74815276.31,10883122.95,11225608.22,21226705.79,43335436.96,6704705.64,25423831.25,43335436.96
4,300713,2015,1,201501,0.0,98906.88,29754.94,0.0,965070.7,365315.36,...,542524.6,10946882.459999999,11357310.25,10041388.88,2314640.15,18939623.98,31295653.01,4465838.65,24334944.02,31952575.9
